In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!git clone https://github.com/gagan-iitb/DS-503

Cloning into 'DS-503'...
remote: Enumerating objects: 605, done.
remote: Counting objects: 100% (605/605), done.
remote: Compressing objects: 100% (554/554), done.
remote: Total 605 (delta 335), reused 65 (delta 30), pack-reused 0
Receiving objects: 100% (605/605), 78.20 MiB | 22.99 MiB/s, done.
Resolving deltas: 100% (335/335), done.


In [ ]:
df = pd.read_csv('/content/DS-503/Assignments/Assignment2-CollabFiltering/Training_set.csv')

In [ ]:
df

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,1,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,2,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
3,3,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,4,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,...,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,99.00,99.00,99.00,99.00,-7.86,99.00,-9.81,6.21,99.00,99.00,99.00,99.00,-7.67,99.00,-7.86,-0.39,-7.33,6.12,-8.50,-0.39,-8.93,99.00,-6.94,99.00,99.00,-6.89,3.59,3.35,-7.82,99.00,-7.82,6.41,99.00,-6.89,3.64,-7.48,99.00,99.00,-6.89,...,-4.66,-0.39,99.00,99.00,-0.58,-7.62,99.00,-6.80,1.60,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.73,99.00,99.00,99.00,99.00,99.00,-0.49,99.00,99.00,99.00,4.22,99.00
19996,19996,7.57,8.93,1.55,3.06,0.49,2.18,8.98,1.60,-0.49,8.79,6.94,9.03,-0.44,8.79,5.68,-1.07,-0.24,8.25,2.23,2.23,8.59,8.79,8.93,8.93,3.93,8.79,9.03,-1.02,9.13,-0.24,8.01,6.02,4.03,8.83,8.20,8.35,9.13,8.88,1.50,...,8.54,-0.63,3.79,-0.49,5.73,-1.02,-0.10,4.42,7.91,3.54,8.59,6.12,0.10,-6.36,6.41,9.03,2.14,2.14,2.14,2.09,8.88,1.07,8.88,9.22,3.35,3.35,8.30,5.63,8.93,2.62,8.83,8.83,8.83,0.00,-1.31,9.03,9.08,-0.78,-2.72,2.52
19997,19997,8.93,-9.17,-4.66,-9.71,8.83,8.88,8.35,8.54,4.85,3.79,1.31,4.66,-4.42,5.58,-2.86,-2.57,2.04,7.52,1.50,8.98,1.26,3.83,5.63,4.85,9.03,5.68,8.74,9.17,9.17,0.39,2.09,-0.83,-9.56,6.21,8.93,8.40,2.91,5.73,8.88,...,7.82,8.69,-8.35,0.87,3.20,8.45,2.96,8.79,3.45,4.85,5.19,6.75,8.98,-9